# Исследование Casino players cashout

Данные взяты из "Metabase",таблица "Affiliate report". Интервал апрель-сентябрь 2020. Действия игроков. Предобработка на SQL не выполнялась. Нужно предобработать и исследовать данные, дать описание игрока который делает кэшаут и не делает для отдела продаж приложив статистику.

## Оглавление:

1. [Загрузка и просмотр данных](#chapter1)
2. [Предобработка и просмотр данных](#chapter2)<br>
3. [Описание Данных](#chapter3)
4. [Группировка и категоризация данных](#chapter4)
5. [Исследование данных](#chapter5)
6. [Общий вывод](#chapter6)

### 1. Загрузка и просмотр данных
<a class="anchor" id="chapter1"></a>

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import io
import matplotlib.pyplot as plt

In [16]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)
print(pd.get_option("display.max_rows"))
print(pd.get_option("display.max_columns"))

100
30


In [48]:
df = pd.read_csv('datasets/au_apr_sep_aff.csv')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135096 entries, 0 to 135095
Data columns (total 27 columns):
ID                           135096 non-null int64
Hash                         135096 non-null object
Date At                      135096 non-null object
Partner ID                   135096 non-null int64
Partner Name                 135095 non-null object
Brand ID                     135096 non-null int64
Brand Name                   135096 non-null object
Strategy                     135096 non-null object
Campaign ID                  135096 non-null int64
Campaign Name                135096 non-null object
Player Country               135096 non-null object
Registrations Count          135096 non-null int64
Currency                     86048 non-null object
First Deposits Count         86048 non-null float64
First Deposits Sum           86048 non-null float64
Deposits Count               86048 non-null float64
Deposits Sum                 86048 non-null float64
Cash Outs C

In [61]:
df

,ID,Date At,Partner ID,Partner Name,Brand ID,Brand Name,Strategy,Campaign ID,Campaign Name,Player Country,Registrations Count,Currency,First Deposits Count,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Additional Deductions Sum,Real Ngr,Created At,Updated At,Player ID
0,100508071706619019,2020-09-17T03:00:00,84181,WryReliableComalny,99,Getslots,CPA 110,99071,Getslots AU AN ORG,AU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-17T12:11:27,2020-09-18T23:58:29,5504932
1,100508071706619032,2020-09-17T03:00:00,84181,WryReliableComalny,99,Getslots,CPA 130,101289,Getslots AU 130 I org,AU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-17T12:11:27,2020-09-18T23:58:29,5504916
2,100508071706619035,2020-09-17T03:00:00,84181,WryReliableComalny,99,Getslots,CPA 130,101289,Getslots AU 130 I org,AU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-17T12:11:27,2020-09-18T23:58:29,5504959
3,100508071706619043,2020-09-17T03:00:00,84181,WryReliableComalny,99,Getslots,CPA iOS 06,93298,GSLOTS AU NEW,AU,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-17T12:11:27,2020-09-17T13:09:17,5504963
4,100508071707473163,2020-09-17T03:00:00,84726,Silver Maple Solutions,99,Getslots,NGR,91512,MAMA,AU,0,AUD,0.0,0.0,1.0,20.0,0.0,0.0,59.14,24.14,15.00,0.0,20.00,2020-09-17T12:11:28,2020-09-18T23:58:32,4554390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135091,991408124614190987,2020-09-17T03:00:00,86027,AcidBro,99,Getslots,CPA 110,98235,Getslots app,AU,0,AUD,0.0,0.0,1.0,25.0,0.0,0.0,8.00,0.88,0.00,0.0,7.12,2020-09-18T02:12:24,2020-09-18T23:59:07,5247918
135092,991408124615645128,2020-09-17T03:00:00,86056,SpicyCasinos,99,Getslots,NGR,94509,Default NGR,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.30,0.00,0.0,-0.30,2020-09-18T02:12:25,2020-09-18T23:59:09,5500981
135093,991408124622813177,2020-09-17T03:00:00,86522,CalMezh,99,Getslots,CPA iOS 06,95900,Getslots 120,AU,0,AUD,0.0,0.0,1.0,15.0,0.0,0.0,86.75,71.80,0.00,0.0,14.95,2020-09-18T02:12:32,2020-09-18T23:59:29,5134536
135094,991408124638551921,2020-09-17T03:00:00,87199,MS Experts s.r.o.,99,Getslots,NGR,96647,Default NGR,AU,1,AUD,0.0,0.0,0.0,0.0,0.0,0.0,2.50,0.15,2.45,0.0,-0.10,2020-09-18T02:12:47,2020-09-18T23:59:52,5511007


### 2. Предобработка и просмотр данных 
<a class="anchor" id="chapter2"></a>

In [51]:
df.drop(columns=['Hash'], inplace=True)

In [37]:
df['Player ID'].value_counts()

86048

Посмотрим на уников плееров с NaN , 1 ли ID или бывает несколько строк на 1 плеера

In [64]:
df.query('`Real Ngr` == "NaN"').sort_values(by='Date At')['Player ID'].value_counts().mean()

1.0

In [67]:
df.query('`Real Ngr` == "NaN"')['Player ID'].count()

49048

По средней видим 1 плеер ID с NaN 1 строка, полагаю это игроки без депов и активности, их удалим, таких 50 000 из 135 000

In [68]:
df.dropna(inplace=True)

Посмотрим на нашу таблицу с кешаут больше нуля и отсортируем ее по плеерам и месяцам

In [233]:
df.query('`Player ID` == 3857272').sort_values(by='Date At')

,ID,Date At,Partner ID,Partner Name,Brand ID,Brand Name,Strategy,Campaign ID,Campaign Name,Player Country,Registrations Count,Currency,First Deposits Count,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Additional Deductions Sum,Real Ngr,Created At,Updated At,Player ID
88529,386207795030518944,2020-04-16T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,50.0,1.0,50.0,0.0,0.0,899.80,849.80,0.00,0.0,50.00,2020-09-14T10:53:48,2020-09-16T14:22:03,3857272
88650,386407795030442163,2020-04-18T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,1.0,30.0,0.0,0.0,242.80,212.80,0.00,0.0,30.00,2020-09-14T10:53:48,2020-09-16T14:22:03,3857272
69091,265877798403011787,2020-05-26T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,1.0,25.0,0.0,0.0,38.50,13.50,0.00,0.0,25.00,2020-09-14T11:47:25,2020-09-17T16:50:00,3857272
71032,265907798402768269,2020-05-29T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,1.0,25.0,0.0,0.0,57.00,32.60,0.00,0.0,24.40,2020-09-14T11:47:24,2020-09-17T16:50:00,3857272
71585,265917798403848528,2020-05-30T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,2.0,40.0,0.0,0.0,1275.75,1227.29,8.00,0.0,40.46,2020-09-14T11:47:25,2020-09-17T16:50:00,3857272
15109,200917792755700230,2020-06-08T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,49.95,16.10,33.80,0.0,0.05,2020-09-14T10:17:39,2020-09-17T19:51:56,3857272
15807,200927792755539162,2020-06-09T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,2.0,50.0,0.0,0.0,118.66,69.01,0.00,0.0,49.65,2020-09-14T10:17:39,2020-09-17T19:51:56,3857272
24338,213467788326145341,2020-07-08T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,2.0,100.0,0.0,0.0,505.85,405.61,0.00,0.0,100.24,2020-09-14T09:07:15,2020-09-14T09:51:04,3857272
28030,213507788325430546,2020-07-12T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,4.0,175.0,2.0,0.0,6327.15,7652.83,0.00,0.0,-1325.68,2020-09-14T09:07:14,2020-09-14T09:51:03,3857272
29278,213517788324723303,2020-07-13T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,0,AUD,0.0,0.0,0.0,0.0,4.0,0.0,292.50,191.75,0.00,0.0,100.75,2020-09-14T09:07:13,2020-09-14T09:51:05,3857272


In [135]:
(df.sort_values(by=['Player ID','Date At'])
)

,ID,Date At,Partner ID,Partner Name,Brand ID,Brand Name,Strategy,Campaign ID,Campaign Name,Player Country,Registrations Count,Currency,First Deposits Count,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Additional Deductions Sum,Real Ngr,Created At,Updated At,Player ID
88324,356307794999171454,2020-04-15T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,20.0,1.0,20.0,0.0,0.0,75.80,56.15,0.00,0.0,19.65,2020-09-14T10:53:18,2020-09-16T14:22:00,3845721
88326,356307794999171469,2020-04-15T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,20.0,2.0,40.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,2020-09-14T10:53:18,2020-09-16T14:22:00,3846729
88328,356307794999171481,2020-04-15T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,20.0,1.0,20.0,0.0,0.0,20.50,0.50,0.00,0.0,20.00,2020-09-14T10:53:18,2020-09-16T14:22:00,3847007
88331,356307794999171488,2020-04-15T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,100.0,1.0,100.0,0.0,0.0,275.00,75.00,100.00,0.0,100.00,2020-09-14T10:53:18,2020-09-16T14:22:00,3847801
88333,356307794999171493,2020-04-15T03:00:00,82464,Geek Team,99,Getslots,CPA,87195,Getslots,AU,1,AUD,1.0,15.0,1.0,15.0,6.0,0.0,172.50,202.60,0.00,0.0,-30.10,2020-09-14T10:53:18,2020-09-16T14:22:00,3848069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8999,181299305381806214,2020-09-30T03:00:00,82469,Union Team,99,Getslots,CPA 07,98363,Getslots web 100,AU,1,AUD,1.0,50.0,2.0,100.0,0.0,0.0,242.30,142.30,0.00,0.0,100.00,2020-10-01T03:00:12,2020-10-05T14:36:36,5650343
93717,666709309176030624,2020-09-30T03:00:00,84181,WryReliableComalny,99,Getslots,CPA 009,100473,Getslots AU 50 org an,AU,1,AUD,1.0,50.0,3.0,100.0,0.0,0.0,655.80,469.50,88.75,0.0,97.55,2020-10-01T04:00:30,2020-10-05T14:39:39,5650798
93730,666709309186195615,2020-09-30T03:00:00,85405,Volkov Team,99,Getslots,CPA 009,100465,Getslots 50 AU 603 an,AU,1,AUD,1.0,25.0,1.0,25.0,0.0,0.0,89.81,127.05,0.00,0.0,-37.24,2020-10-01T04:00:40,2020-10-05T14:40:29,5650844
93738,666709309238476330,2020-09-30T03:00:00,88081,Carrington Oliver John ltd,99,Getslots,NGR,98702,Default NGR,AU,1,AUD,0.0,0.0,0.0,0.0,0.0,0.0,9.24,6.60,2.70,0.0,-0.06,2020-10-01T04:01:30,2020-10-05T14:55:26,5650849


### 3. Описание Данных
<a class="anchor" id="chapter3"></a>

Опишем наши данные

In [246]:
df.query('`Player ID` == 5129537').sort_values(by='Date At')


,ID,Date At,Partner ID,Partner Name,Brand ID,Brand Name,Strategy,Campaign ID,Campaign Name,Player Country,Registrations Count,Currency,First Deposits Count,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Additional Deductions Sum,Real Ngr,Created At,Updated At,Player ID
82847,299407785311216888,2020-09-09T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,1.0,25.0,1.0,25.0,0.0,0.0,275.95,353.55,25.0,0.0,-102.60,2020-09-14T08:19:19,2020-09-14T08:19:19,5129537
83893,299417785311951514,2020-09-10T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,79.00,24.40,0.0,0.0,54.60,2020-09-14T08:19:20,2020-09-14T08:19:20,5129537
34333,234088147268653025,2020-09-18T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,3.50,0.60,0.0,0.0,2.90,2020-09-18T08:12:29,2020-09-19T23:59:49,5129537
83,104048678743650447,2020-09-24T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,-45.1,0.0,45.10,2020-09-24T05:00:03,2020-10-05T14:35:22,5129537
88270,349009052456773653,2020-09-28T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,5.50,0.00,0.0,0.0,5.50,2020-09-28T08:00:03,2020-10-05T14:35:22,5129537
88417,364709154378092399,2020-09-29T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,29.50,30.70,0.0,0.0,-1.20,2020-09-29T11:00:03,2020-10-05T14:35:22,5129537
73284,272499248749826316,2020-09-30T03:00:00,82464,Geek Team,99,Getslots,CPA 110,97540,Getslots 110 AU,AU,0,AUD,0.0,0.0,0.0,0.0,0.0,0.0,21.00,1.25,0.0,0.0,19.75,2020-09-30T12:00:03,2020-10-05T14:35:22,5129537


In [185]:
df_filter = df[['Deposits Count', 'Deposits Sum', 'Cash Outs Count', 'Cash Outs Sum','Bets Sum','Wins Sum','Bonus Issues Sum','Additional Deductions Sum']].describe()

,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Additional Deductions Sum
count,86048.000000,86048.000000,86048.000000,86048.000000,86048.000000,86048.000000,86048.000000,86048.000000
mean,1.337393,88.834844,0.386831,30.256112,1468.589428,1400.541004,13.565320,-0.725418
std,1.986803,399.134787,1.707837,617.948338,8635.630669,8635.742781,73.233566,61.901045
min,0.000000,0.000000,0.000000,-23000.000000,0.000000,0.000000,-10468.800000,-10732.040000
25%,0.000000,0.000000,0.000000,0.000000,29.440000,9.700000,0.000000,0.000000
50%,1.000000,25.000000,0.000000,0.000000,175.000000,118.625000,0.000000,0.000000
75%,2.000000,75.000000,0.000000,0.000000,759.697500,658.255000,18.735000,0.000000
max,42.000000,55000.000000,164.000000,90000.000000,784033.000000,780752.007451,2538.000000,620.670000


In [250]:
print("all string count =",df['Cash Outs Sum'].count())
print("all cashouts string sum =",df['Cash Outs Sum'].sum())
print('cashouts < 0 counts =',df.query('`Cash Outs Sum` < 0')['Cash Outs Sum'].count())
print('cashouts < 0 sum =',df.query('`Cash Outs Sum` < 0')['Cash Outs Sum'].sum())
print('cashouts > 0 counts =',df.query('`Cash Outs Sum` > 0')['Cash Outs Sum'].count())
print('cashouts > 0 sum =',df.query('`Cash Outs Sum` > 0')['Cash Outs Sum'].sum())

all string count = 86048
all cashouts string sum = 2603477.90011242
cashouts < 0 counts = 782
cashouts < 0 sum = -806653.4
cashouts > 0 counts = 3971
cashouts > 0 sum = 3410131.30011242


In [248]:
print('sum deps $ =',df['Deposits Sum'].sum())
print('sum cashout $ =',df['Cash Outs Sum'].sum())

sum deps $ = 7644060.682279689
sum cashout $ = 2603477.90011242


Посмотрим столбцы Additional Deductions Sum и Bonus Issues Sum

In [229]:
print('`Additional Deductions Sum` < 0 count =',df.query('`Additional Deductions Sum` < 0')['Additional Deductions Sum'].count())
print('`Additional Deductions Sum` < 0 sum =',df.query('`Additional Deductions Sum` < 0')['Additional Deductions Sum'].sum())
print('`Additional Deductions Sum` > 0 count =',df.query('`Additional Deductions Sum` > 0')['Additional Deductions Sum'].count())
print('`Additional Deductions Sum` > 0 sum =',df.query('`Additional Deductions Sum` > 0')['Additional Deductions Sum'].sum())

`Additional Deductions Sum` < 0 count = 54
`Additional Deductions Sum` < 0 sum = -63196.33
`Additional Deductions Sum` > 0 count = 3
`Additional Deductions Sum` > 0 sum = 775.6


In [231]:
print('`Bonus Issues Sum` < 0 count =',df.query('`Bonus Issues Sum` < 0')['Bonus Issues Sum'].count())
print('`Bonus Issues Sum` < 0 sum =',df.query('`Bonus Issues Sum` < 0')['Bonus Issues Sum'].sum())
print('`Bonus Issues Sum` > 0 count =',df.query('`Bonus Issues Sum` > 0')['Bonus Issues Sum'].count())
print('`Bonus Issues Sum` > 0 sum =',df.query('`Bonus Issues Sum` > 0')['Bonus Issues Sum'].sum())

`Bonus Issues Sum` < 0 count = 9807
`Bonus Issues Sum` < 0 sum = -206873.61004134
`Bonus Issues Sum` > 0 count = 32234
`Bonus Issues Sum` > 0 sum = 1374142.2368931598


In [241]:
df.shape

(86048, 26)

### Вывод

Мы видим столюцы с депами, выводами , бонусами, датами и ID по каждому игроку без группировки. У нас минусовые, значения в Кэшауте, это отмененные выводы, итоговая сумма по тоталу берется.
Additional Deductions Sum - столбец с доп отчислениями и ручными корректировками, он имеет менне процента от валовой выручки его удаляем.
Bonus Issues Sum - есть минусовые бонусы и плюсовые, плюсовые имеют большой тотал 1,4 млн, около 20% от валовой выручки.

### 4. Группировка данных и категоризация
<a class="anchor" id="chapter4"></a>

Посмотрим на типового игрока который делает, кэшаут

In [265]:
df.query('`Player ID` == 3900150').sort_values(by='Date At')

,ID,Date At,Partner ID,Partner Name,Brand ID,Brand Name,Strategy,Campaign ID,Campaign Name,Player Country,Registrations Count,Currency,First Deposits Count,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Real Ngr,Created At,Updated At,Player ID
89140,386707795062899180,2020-04-21T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,1,AUD,1.0,200.0,1.0,200.0,3.0,1000.0,6973.00,7623.00,150.0,-800.00,2020-09-14T10:54:19,2020-09-16T14:23:18,3900150
89257,386807795039591642,2020-04-22T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,0.0,0.0,8.0,2000.0,16050.00,18021.00,30.0,-2001.00,2020-09-14T10:53:57,2020-09-16T14:23:18,3900150
89431,386907795040446467,2020-04-23T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,2.0,800.0,0.0,0.0,2715.00,1916.00,0.0,799.00,2020-09-14T10:53:58,2020-09-16T14:23:18,3900150
90755,387607795040645406,2020-04-30T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,1.0,200.0,0.0,0.0,281.25,79.50,0.0,201.75,2020-09-14T10:53:58,2020-09-16T13:14:49,3900150
64477,265797798409096820,2020-05-18T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,2.0,400.0,1.0,400.0,6788.90,6788.95,0.0,-0.05,2020-09-14T11:47:30,2020-09-17T16:50:46,3900150
64966,265807798412205913,2020-05-19T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,1.0,200.0,0.0,0.0,776.00,577.40,0.0,198.60,2020-09-14T11:47:33,2020-09-17T16:50:46,3900150
72669,265927798513520939,2020-05-31T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,2.0,150.0,0.0,0.0,5169.50,5019.50,0.0,150.00,2020-09-14T11:49:10,2020-09-14T11:49:10,3900150
10430,200847792857222622,2020-06-01T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,3.0,200.0,0.0,0.0,839.75,638.20,0.0,201.55,2020-09-14T10:19:16,2020-09-17T19:58:37,3900150
14018,200897792856538040,2020-06-06T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,1.0,80.0,0.0,0.0,450.00,370.00,0.0,80.00,2020-09-14T10:19:15,2020-09-17T19:58:37,3900150
16803,200937792870846528,2020-06-10T03:00:00,82469,Union Team,99,Getslots,CPA,87197,Getslots,AU,0,AUD,0.0,0.0,2.0,75.0,1.0,250.0,665.50,842.10,0.0,-176.60,2020-09-14T10:19:29,2020-09-17T19:58:51,3900150


Сведем фильтрованные данные которые нам интересны, в отдельный df_filter 

In [410]:
df_filter = df[['Date At','Player ID','First Deposits Sum','Deposits Count', 'Deposits Sum', 'Cash Outs Count', 'Cash Outs Sum','Bets Sum','Wins Sum','Bonus Issues Sum']].copy()
df_filter

,Date At,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum
4,2020-09-17T03:00:00,4554390,0.0,1.0,20.0,0.0,0.0,59.14,24.14,15.00
5,2020-09-17T03:00:00,5004487,0.0,0.0,0.0,0.0,0.0,143.95,93.70,0.00
7,2020-09-17T03:00:00,5504921,50.0,1.0,50.0,0.0,0.0,181.30,81.50,50.00
9,2020-09-17T03:00:00,5504961,50.0,1.0,50.0,0.0,0.0,195.00,95.20,50.00
10,2020-09-17T03:00:00,5504953,50.0,3.0,200.0,0.0,0.0,2682.20,2316.88,166.25
...,...,...,...,...,...,...,...,...,...,...
135091,2020-09-17T03:00:00,5247918,0.0,1.0,25.0,0.0,0.0,8.00,0.88,0.00
135092,2020-09-17T03:00:00,5500981,0.0,0.0,0.0,0.0,0.0,0.00,0.30,0.00
135093,2020-09-17T03:00:00,5134536,0.0,1.0,15.0,0.0,0.0,86.75,71.80,0.00
135094,2020-09-17T03:00:00,5511007,0.0,0.0,0.0,0.0,0.0,2.50,0.15,2.45


Из df_filter оставим строки с днями, когда игрок играл был активен, либо депал или выводил либо играл

In [411]:
df_filter_act = df_filter.query('`Deposits Sum` != 0 or `Cash Outs Sum` != 0 or `Bets Sum` > 0').copy()

In [412]:
df_filter_act.reset_index(inplace=True,drop=True)
df_filter_act

,Date At,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum
0,2020-09-17T03:00:00,4554390,0.0,1.0,20.0,0.0,0.0,59.14,24.14,15.00
1,2020-09-17T03:00:00,5004487,0.0,0.0,0.0,0.0,0.0,143.95,93.70,0.00
2,2020-09-17T03:00:00,5504921,50.0,1.0,50.0,0.0,0.0,181.30,81.50,50.00
3,2020-09-17T03:00:00,5504961,50.0,1.0,50.0,0.0,0.0,195.00,95.20,50.00
4,2020-09-17T03:00:00,5504953,50.0,3.0,200.0,0.0,0.0,2682.20,2316.88,166.25
...,...,...,...,...,...,...,...,...,...,...
74937,2020-09-17T03:00:00,5374244,0.0,1.0,70.0,0.0,0.0,414.25,292.10,52.50
74938,2020-09-17T03:00:00,5247918,0.0,1.0,25.0,0.0,0.0,8.00,0.88,0.00
74939,2020-09-17T03:00:00,5134536,0.0,1.0,15.0,0.0,0.0,86.75,71.80,0.00
74940,2020-09-17T03:00:00,5511007,0.0,0.0,0.0,0.0,0.0,2.50,0.15,2.45


### Вывод

Данные сгруппированы теперь можно их исследовать

### 5. Исследование данных
<a class="anchor" id="chapter5"></a>

#### Теперь использую наши вопросы

- Сколько плюсовых дней до кешаута?
- Какая обычная сумма (разница между wins - bets), предшествует кешауту? 
- Зависимость от суммы депозитов игрока (будут разные показатели для тех, кто депает много и мало).

Создадим столбец с кэшаут c ключом где событие вывода будет равно 1, а событие отмены вывода равно -1

In [480]:
df_filter_act['Cashout_key_plus'] = (df_filter_act['Cash Outs Sum'] > 0)*1
df_filter_act['Cashout_key_minus'] = (df_filter_act['Cash Outs Sum'] < 0)*(-1)

In [481]:
df_filter_act['Cashout_key'] = df_filter_act['Cashout_key_plus'] + df_filter_act['Cashout_key_minus'] 

In [482]:
df_filter_act.drop(columns=['Cashout_key_plus','Cashout_key_minus'], inplace=True)

In [483]:
df_filter_act.query('Cashout_key == -1')

,Date At,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Cashout_key,income_player
536,2020-05-03T03:00:00,3991790,0.0,0.0,0.0,-1.0,-50.0,506.29,490.83,0.00,-1,-15.46
547,2020-05-03T03:00:00,3916119,0.0,0.0,0.0,-1.0,-3000.0,383.75,93.70,0.00,-1,-290.05
555,2020-05-03T03:00:00,3936280,0.0,2.0,100.0,-3.0,-100.0,276.24,276.10,0.00,-1,-0.14
580,2020-05-03T03:00:00,3985651,0.0,0.0,0.0,-1.0,-200.0,851.50,811.35,0.00,-1,-40.15
586,2020-05-03T03:00:00,3989914,0.0,0.0,0.0,-7.0,-150.0,253.90,203.92,0.25,-1,-49.98
...,...,...,...,...,...,...,...,...,...,...,...,...
73408,2020-07-04T03:00:00,4553256,0.0,0.0,0.0,-1.0,-180.0,618.00,434.65,0.00,-1,-183.35
73459,2020-07-04T03:00:00,4547858,0.0,0.0,0.0,-1.0,-150.0,197.00,47.28,0.00,-1,-149.72
73590,2020-07-05T03:00:00,4548673,0.0,0.0,0.0,-1.0,-900.0,35.00,6.50,0.00,-1,-28.50
73681,2020-07-06T03:00:00,4559214,0.0,0.0,0.0,-1.0,-850.0,0.00,0.00,0.00,-1,0.00


Поменяем каунты по кэшаутам на минусовые, когда у нас была отмена списания

In [484]:
df_filter_act['Cash Outs Count'] = df_filter_act['Cash Outs Count'] * df_filter_act['Cashout_key']

Проверим статистику по клиенту который делал кэшаут

In [485]:
df_filter_act.query('`Player ID` == 3900150').sort_values(by='Date At')

,Date At,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Cashout_key,income_player
49491,2020-04-21T03:00:00,3900150,200.0,1.0,200.0,3.0,1000.0,6973.00,7623.00,150.0,1,650.00
49542,2020-04-22T03:00:00,3900150,0.0,0.0,0.0,8.0,2000.0,16050.00,18021.00,30.0,1,1971.00
49614,2020-04-23T03:00:00,3900150,0.0,2.0,800.0,0.0,0.0,2715.00,1916.00,0.0,0,-799.00
50303,2020-04-30T03:00:00,3900150,0.0,1.0,200.0,0.0,0.0,281.25,79.50,0.0,0,-201.75
35728,2020-05-18T03:00:00,3900150,0.0,2.0,400.0,1.0,400.0,6788.90,6788.95,0.0,1,0.05
35974,2020-05-19T03:00:00,3900150,0.0,1.0,200.0,0.0,0.0,776.00,577.40,0.0,0,-198.60
40144,2020-05-31T03:00:00,3900150,0.0,2.0,150.0,0.0,0.0,5169.50,5019.50,0.0,0,-150.00
5556,2020-06-01T03:00:00,3900150,0.0,3.0,200.0,0.0,0.0,839.75,638.20,0.0,0,-201.55
7540,2020-06-06T03:00:00,3900150,0.0,1.0,80.0,0.0,0.0,450.00,370.00,0.0,0,-80.00
9063,2020-06-10T03:00:00,3900150,0.0,2.0,75.0,1.0,250.0,665.50,842.10,0.0,1,176.60


Посмотрим на распределение по количеству кешаутов при разных условиях `Bets Sum` <= `Wins Sum` и `Bets Sum` > `Wins Sum`

In [486]:
print('total cashout count =',df_filter_act.query('`Cashout_key` == 1')['Cashout_key'].sum())
print('total cashout sum =',df_filter_act.query('`Cashout_key` == 1')['Cash Outs Sum'].sum())


total cashout count = 3971
total cashout sum = 3410131.30011242


In [487]:
print('cashout `Bets Sum` <= `Wins Sum` count = ',df_filter_act.query('`Bets Sum` <= `Wins Sum` and `Cashout_key` == 1')['Cashout_key'].sum())
print('cashout `Bets Sum` <= `Wins Sum` sum =',df_filter_act.query('`Bets Sum` <= `Wins Sum` and `Cashout_key` == 1')['Cash Outs Sum'].sum())

cashout `Bets Sum` <= `Wins Sum` count =  2810
cashout `Bets Sum` <= `Wins Sum` sum = 2603604.13231116


In [488]:
print("cashout `Bets Sum` > `Wins Sum` count =",df_filter_act.query('`Bets Sum` > `Wins Sum` and `Cashout_key` == 1')['Cashout_key'].sum())
print("cashout `Bets Sum` > `Wins Sum` sum =",df_filter_act.query('`Bets Sum` > `Wins Sum` and `Cashout_key` == 1')['Cash Outs Sum'].sum())

cashout `Bets Sum` > `Wins Sum` count = 1161
cashout `Bets Sum` > `Wins Sum` sum = 806527.16780126


In [489]:
2603/3410

0.7633431085043988

#### Мы видим распределение по обороту денег 75% объема выводят в день когда выйгрыш больше или равен тратам, 25% выводят в день, когда выйгрыш меньше затрат игрока.

Создадим столбец "wins_key"с ключами по данным когда выйгрыш был больше или равен проигрышу и столбец с размером выйгрыша

In [490]:
df_filter_act['wins_key'] = (df_filter_act['Wins Sum'] >= df_filter_act['Bets Sum'])*1

In [491]:
df_filter_act['income_player'] = df_filter_act['Wins Sum'] - df_filter_act['Bets Sum']

In [492]:
df_filter_act.query('`Bets Sum` <= `Wins Sum`')

,Date At,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Cashout_key,income_player,wins_key
5,2020-09-17T03:00:00,4558618,0.0,0.0,0.0,0.0,0.0,337.50,575.50,0.00,0,238.00,1
12,2020-09-17T03:00:00,4862849,25.0,1.0,25.0,1.0,150.0,7000.50,7121.55,25.00,1,121.05,1
28,2020-09-14T03:00:00,4416610,0.0,2.0,175.0,0.0,0.0,958.00,964.55,0.00,0,6.55,1
30,2020-09-14T03:00:00,4624435,0.0,0.0,0.0,0.0,0.0,146.00,184.90,0.00,0,38.90,1
59,2020-09-24T03:00:00,4676921,0.0,7.0,525.0,2.0,1600.0,5030.35,6006.04,0.00,1,975.69,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74880,2020-09-15T03:00:00,5492423,25.0,1.0,25.0,0.0,0.0,94.00,94.65,25.00,0,0.65,1
74890,2020-09-17T03:00:00,5503455,25.0,1.0,25.0,0.0,0.0,94.50,120.70,30.22,0,26.20,1
74893,2020-09-24T03:00:00,5487976,0.0,3.0,150.0,0.0,0.0,0.00,0.00,0.00,0,0.00,1
74905,2020-09-20T03:00:00,4994399,0.0,1.0,100.0,0.0,0.0,3809.60,4170.70,40.00,0,361.10,1


In [479]:
df_filter_act['Player ID'].value_counts().count()

21479

Соберем в отдельный отчет инфу по депам и кешаутам по каждому плееру

In [519]:
player_report = (df_filter_act
    .sort_values(by=['Player ID','Date At'])
    .groupby(['Player ID'])
    .agg({'First Deposits Sum':['sum'],'Deposits Count' : ['sum'],'Deposits Sum' : ['sum'],'Cash Outs Count' : ['sum'], 'Cash Outs Sum':['sum'],'Bets Sum':['sum'],'Wins Sum':['sum'],'Bonus Issues Sum':['sum'],'income_player':['sum'],'Cashout_key':['sum'],'wins_key':['sum']})
    .round(2)
    .reset_index()
)
player_report

,Player ID,First Deposits Sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,income_player,Cashout_key,wins_key
,,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
0,3845721,20.0,1.0,20.0,0.0,0.0,75.80,56.15,0.00,-19.65,0,0
1,3846729,20.0,2.0,40.0,0.0,0.0,0.00,0.00,0.00,0.00,0,1
2,3847007,20.0,1.0,20.0,0.0,0.0,20.50,0.50,0.00,-20.00,0,0
3,3847801,100.0,1.0,100.0,0.0,0.0,275.00,75.00,100.00,-200.00,0,0
4,3848069,15.0,1.0,15.0,0.0,0.0,188.25,215.50,0.00,27.25,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
21474,5650343,50.0,2.0,100.0,0.0,0.0,242.30,142.30,0.00,-100.00,0,0
21475,5650798,50.0,3.0,100.0,0.0,0.0,655.80,469.50,88.75,-186.30,0,0
21476,5650844,25.0,1.0,25.0,0.0,0.0,89.81,127.05,0.00,37.24,0,1


In [534]:
player_report.columns = ['Player ID','FTD_sum','Deposits Count', 'Deposits Sum', 'Cash Outs Count', 'Cash Outs Sum','Bets Sum','Wins Sum','Bonus Issues Sum','Income_player','Cashout_days','Wins_days']

In [535]:
player_report.head()

,Player ID,FTD_sum,Deposits Count,Deposits Sum,Cash Outs Count,Cash Outs Sum,Bets Sum,Wins Sum,Bonus Issues Sum,Income_player,Cashout_days,Wins_days
0,3845721,20.0,1.0,20.0,0.0,0.0,75.80,56.15,0.0,-19.65,0,0
1,3846729,20.0,2.0,40.0,0.0,0.0,0.00,0.00,0.0,0.00,0,1
2,3847007,20.0,1.0,20.0,0.0,0.0,20.50,0.50,0.0,-20.00,0,0
3,3847801,100.0,1.0,100.0,0.0,0.0,275.00,75.00,100.0,-200.00,0,0
4,3848069,15.0,1.0,15.0,0.0,0.0,188.25,215.50,0.0,27.25,0,1


Теперь посмотрим средние по player_report, когда cashout был выше нуля

In [537]:
g = player_report.query('Cashout_days != 0').shape[0]
t = player_report.shape[0]
n = g / t

print('all players =',t)
print('all cashoup players =',g)
print('share of cashout players in total(%) =',round(n,2))

all players = 21479
all cashoup players = 1040
share of cashout players in total(%) = 0.05


In [538]:
player_report.query('Cashout_days > 0 and Wins_days > 0')['Cash Outs Sum'].sum()

2538522.88

Теперь выведем средние по игрокам которые не делают и делают кэшаут

In [542]:
player_report.query('Cashout_days == 0').mean().round(2)

Player ID           4834446.31
FTD_sum                  34.51
Deposits Count            3.71
Deposits Sum            203.07
Cash Outs Count           0.17
Cash Outs Sum             2.74
Bets Sum               3413.54
Wins Sum               3186.45
Bonus Issues Sum         48.67
Income_player          -227.09
Cashout_days              0.00
Wins_days                 0.32
dtype: float64

In [546]:
print('Средний инком перед кэшаутом =',df_filter_act.query('Cashout_key != 0 and income_player > 0')['income_player'].mean().round(2))

средний инком перед кэшаутом = 867.13


In [548]:
print('Средний убыток перед кэшаутом =',df_filter_act.query('Cashout_key != 0 and income_player < 0')['income_player'].mean().round(2))

Средний убыток перед кэшаутом = -724.09


In [540]:
player_report.query('Cashout_days != 0 ').mean().round(2)

Player ID           4631452.86
FTD_sum                  45.63
Deposits Count           37.75
Deposits Sum           3359.16
Cash Outs Count          11.48
Cash Outs Sum          2449.44
Bets Sum              54422.89
Wins Sum              53250.06
Bonus Issues Sum        255.69
Income_player         -1172.83
Cashout_days              3.07
Wins_days                 4.30
dtype: float64

In [550]:
print('Средний инком игрока перед кэшаутом =',df_filter_act.query('Cashout_key != 0 and income_player > 0')['income_player'].mean().round(2))

Средний инком игрока перед кэшаутом = 867.13


In [548]:
print('Средний убыток перед кэшаутом =',df_filter_act.query('Cashout_key != 0 and income_player < 0')['income_player'].mean().round(2))

Средний убыток перед кэшаутом = -724.09


Теперь посмотрим на показатели для разных сумм депов

In [557]:
player_report.query('Cashout_days != 0')['Deposits Sum'].describe().round(2)

count      1040.00
mean       3359.16
std        9160.58
min           0.03
25%         250.00
50%         800.00
75%        2826.25
max      203962.85
Name: Deposits Sum, dtype: float64

In [555]:
player_report.query('Cashout_days != 0 and `Deposits Sum` <= 500').mean().round(2)

Player ID           4714044.07
FTD_sum                  32.41
Deposits Count            6.08
Deposits Sum            199.56
Cash Outs Count           5.81
Cash Outs Sum           509.97
Bets Sum               6514.53
Wins Sum               6772.18
Bonus Issues Sum         50.14
Income_player           257.65
Cashout_days              1.29
Wins_days                 2.32
dtype: float64

In [553]:
player_report.query('Cashout_days != 0 and 500 < `Deposits Sum` <= 1000').mean().round(2)

Player ID           4715472.05
FTD_sum                  34.46
Deposits Count            9.35
Deposits Sum            347.59
Cash Outs Count           6.35
Cash Outs Sum           667.76
Bets Sum               9318.67
Wins Sum               9558.64
Bonus Issues Sum         69.44
Income_player           239.98
Cashout_days              1.53
Wins_days                 2.74
dtype: float64

In [556]:
player_report.query('Cashout_days != 0 and 1000 < `Deposits Sum` <= 2000').mean().round(2)

Player ID           4661444.36
FTD_sum                  52.04
Deposits Count           29.44
Deposits Sum           1437.59
Cash Outs Count           9.32
Cash Outs Sum          1442.17
Bets Sum              22000.27
Wins Sum              21867.80
Bonus Issues Sum        169.21
Income_player          -132.47
Cashout_days              2.51
Wins_days                 4.04
dtype: float64

In [566]:
player_report.query('Cashout_days != 0 and 2000 < `Deposits Sum` <= 3000').mean().round(2)

Player ID           4519851.36
FTD_sum                  41.60
Deposits Count           47.35
Deposits Sum           2529.00
Cash Outs Count          13.24
Cash Outs Sum          2545.25
Bets Sum              37129.72
Wins Sum              36992.01
Bonus Issues Sum        209.12
Income_player          -137.71
Cashout_days              3.82
Wins_days                 6.10
dtype: float64

In [583]:
player_report.query('Cashout_days != 0 and 3000 < `Deposits Sum` <= 5000').mean().round(2)

Player ID           4473275.74
FTD_sum                  56.31
Deposits Count           66.47
Deposits Sum           3807.01
Cash Outs Count          13.49
Cash Outs Sum          2702.87
Bets Sum              58511.84
Wins Sum              57211.22
Bonus Issues Sum        286.78
Income_player         -1300.63
Cashout_days              3.59
Wins_days                 4.69
dtype: float64

In [584]:
player_report.query('Cashout_days != 0 and 5000 < `Deposits Sum` <= 10000').mean().round(2)

Player ID           4458586.49
FTD_sum                  61.09
Deposits Count           92.12
Deposits Sum           7307.17
Cash Outs Count          20.89
Cash Outs Sum          5239.93
Bets Sum             129006.00
Wins Sum             126532.44
Bonus Issues Sum        572.59
Income_player         -2473.56
Cashout_days              6.67
Wins_days                 7.25
dtype: float64

In [585]:
player_report.query('Cashout_days != 0 and 10000 < `Deposits Sum`').mean().round(2)

Player ID           4437246.94
FTD_sum                  86.78
Deposits Count          156.39
Deposits Sum          23774.20
Cash Outs Count          37.69
Cash Outs Sum         13324.90
Bets Sum             358045.03
Wins Sum             345834.41
Bonus Issues Sum       1379.02
Income_player        -12210.63
Cashout_days              9.78
Wins_days                10.58
dtype: float64

In [581]:
print(' Deposits Sum <= 500:',player_report.query('Cashout_days != 0 and `Deposits Sum` < 500').shape[0])
print(' 500 < Deposits Sum <= 1000 :',player_report.query('Cashout_days != 0 and 500 < `Deposits Sum` <= 1000').shape[0])
print(' 1000 < Deposits Sum <= 2000 :',player_report.query('Cashout_days != 0 and 1000 < `Deposits Sum` <= 2000').shape[0])
print(' 2000 < Deposits Sum <= 3000 :',player_report.query('Cashout_days != 0 and 2000 < `Deposits Sum` <= 3000').shape[0])
print(' 3000 < Deposits Sum <= 5000 :',player_report.query('Cashout_days != 0 and 3000 < `Deposits Sum` <= 5000').shape[0])
print(' 5000 < Deposits Sum <= 10000 :',player_report.query('Cashout_days != 0 and 5000 < `Deposits Sum` <= 10000').shape[0])
print('10000 < Deposits Sum :',player_report.query('Cashout_days != 0 and 10000 < `Deposits Sum`').shape[0])
print(' Deposits Sum total <= 500:',player_report.query('Cashout_days != 0').shape[0])

 Deposits Sum <= 500: 398
 500 < Deposits Sum <= 1000 : 166
 1000 < Deposits Sum <= 2000 : 154
 2000 < Deposits Sum <= 3000 : 72
 3000 < Deposits Sum <= 5000 : 78
 5000 < Deposits Sum <= 10000 : 85
10000 < Deposits Sum : 83
 Deposits Sum total <= 500: 1040


## Общий Вывод
<a class="anchor" id="chapter6"></a>

Описание игрока который не делает кэшаут:

- 95%  доля таких игроков
- 0,3  победных дня в среднем на игрока без кэшаута
- 0    кэшаутов дня на в среднем на игрока
- 0    баксов (0) кэшауты
- 0    средний кэшаут
- 200  баксов cредняя сумма депов от игрока
- 4 депа cреднее число депов от игрока
- 50 баксов средний деп
- -220 баксов средний доход игрока
- 3,5к  средний оборот
- 50  средний бонус
- 35   средний FTD  по игроку

Описание игрока который делает кэшаут:

- 5% доля таких игроков
- 1,5 победных дня в среднем на игрока до кэшаута
- 11 кэшаут дня на в среднем на игрока
- 2400 баксов кэшауты
- 220 баксов средний кэшаут
- 3400 баксов cредняя сумма депов от игрока
- 38 депов cреднее число депов от игрока
- 90 баксов средний деп
- -1200 баксов средний доход игрока
- 55к средний оборот
- 250 средний бонус
- 45 средний FTD по игроку
- 75% игроков выводят в день когда выйгрыш больше или равен расходам
- средний выйгрыш перед выводом 850 доларов
- 25% выводят в день, когда выйгрыш меньше расходов игрока
- средний убыток перед выводом 700 долларов

- 55% игроков с суммой депов до 1'000 долларов, выводят в  +250 долларов
- 15% игроков с суммой депов от 1'000 до 2'000 долларов, выводят в +130 долларов
- 7% игроков с суммой депов от 2'000 до 3'000 долларов, выводят в -140 долларов
- 7% игроков с суммой депов от 3'000 до 5'000 долларов, выводят в -1'300 долларов
- 8% игроков с суммой депов от 5'000 до 10'000 долларов, выводят в -2'500 долларов
- 8% игроков с суммой депов от 10'000 долларов, выводят в -12'000 долларов